<a href="https://colab.research.google.com/github/manasarthak/Emotion-classification-using-physiological-signal/blob/main/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install antropy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from IPython.utils import io
import numpy as np
import collections

from sklearn.preprocessing import StandardScaler,MinMaxScaler,PolynomialFeatures
from sklearn.utils import shuffle
from sklearn import datasets

import scipy.io
from scipy import signal,integrate
from scipy import stats
from scipy.stats import skew,kurtosis,entropy,iqr
from scipy.signal import welch
from scipy.integrate import simps

from pandas.plotting import scatter_matrix
import pandas as pd
from pandas import DataFrame

import matplotlib.pyplot as plt
import math
import antropy as ap

import copy

In [3]:
def load_data(dim):
    if dim=='valence':
        labels_all=np.load('/content/drive/MyDrive/DEAP/valence/' + 'all_valence_labels.npy',allow_pickle=True)
        data_all=np.load('/content/drive/MyDrive/DEAP/valence/' + 'all_valence_data.npy',allow_pickle=True)
        print("Valence :",labels_all.shape,data_all.shape)
    elif dim=='arousal':
        labels_all=np.load('/content/drive/MyDrive/DEAP/arousal/'+'all_arousal_labels.npy',allow_pickle=True)
        data_all=np.load('/content/drive/MyDrive/DEAP/arousal/'+'all_arousal_data.npy',allow_pickle=True)
        print("Arousal: ",labels_all.shape,data_all.shape)
    return labels_all,data_all

In [4]:
labels_all_val,data_all_val=load_data(dim='valence')

Valence : (1280,) (1280, 32, 6400)


In [5]:
labels_all_ar,data_all_ar=load_data(dim='arousal')

Arousal:  (1280,) (1280, 32, 6400)


In [6]:
datax=data_all_val #for convenience as data for valence and arousal is same 

In [7]:
datax

array([[[-4.83214893e-01, -1.74137370e+00, -2.28734619e+00, ...,
          8.61262096e-01,  2.37428590e-01, -1.01922760e-01],
        [-1.94500659e-01, -9.62530098e-01, -1.49152694e+00, ...,
          6.66734248e-01,  4.64355433e-01,  3.05838501e-04],
        [-1.72581175e-01, -1.21764826e+00, -1.20665943e+00, ...,
          3.69157876e-01,  9.92985443e-02, -2.66627271e-01],
        ...,
        [-2.05258526e+00, -1.67593365e+00, -1.38214124e+00, ...,
          1.20531957e+00,  4.14356542e-01,  4.67189581e-01],
        [-4.49033212e-01,  1.05922304e+00,  2.26109791e+00, ...,
          8.78822753e-01,  8.78406757e-02,  3.12358701e-01],
        [-6.28899143e-01,  8.01583086e-01,  1.85301729e+00, ...,
          4.70403218e-01, -2.19645862e-01,  1.65929051e-01]],

       [[ 5.67487705e-01,  1.81006280e-01, -2.80903829e-01, ...,
          1.68276740e-01,  1.61362446e-01, -6.00499905e-02],
        [ 7.88620041e-01,  2.99460907e-01, -5.20917166e-01, ...,
         -4.20272158e-01, -2.69602752e

Statistical/Time Domain Features(Linear and Complex)

In [8]:
def mean(x):
  return (np.mean(x,axis=-1));

def std(data):
    return np.std(data,axis=-1)#standard deviation

def ptp(data):
    return np.ptp(data,axis=-1)#range

def minim(data):
      return np.min(data,axis=-1)

def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)#indices of the minimum value(related to time)

def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): 
      return  np.sqrt(np.mean(data**2,axis=-1)) #root mean square 

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)

def skewness(data):
    return skew(data,axis=-1)#measure of assysmetry of a distribution

def kkurtosis(data):
    return stats.kurtosis(data,axis=-1)#measure of tailedness of a distribution

def calc_entropy(data):
  return np.apply_along_axis(entropy,-1,data);

def calc_IQR(data):
  return np.apply_along_axis(iqr,-1,data);

def hjorth_activity(data):
  return np.var(data,axis=-1);

def hjorth_mobility(data):
  return np.divide(np.std(np.diff(data, axis=-1)),np.std(data, axis=-1));

def hjorth_complexity(data):
  return np.divide(hjorth_mobility(np.diff(data, axis=-1)), hjorth_mobility(data));

def petrosian_fractD(data):#Original code from the `pyrem <https://github.com/gilestrolab/pyrem>`_package by Quentin Geissmann.
  N=data.shape[-1];
  nzc_derivative=ap.num_zerocross(np.diff(data,axis=-1),axis=-1);#counting the number of zero cross in the first derivative of signal
  pfd = np.log10(N) / (np.log10(N) + np.log10(N / (N + 0.4 * nzc_derivative)));
  return pfd;

def katz_fd(data):#Original code from the `mne-features <https://mne.tools/mne-features/>`_package by Jean-Baptiste Schiratti and Alexandre Gramfort.
  dists = np.abs(np.diff(data, axis=-1))
  ll = dists.sum(axis=-1)
  ln = np.log10(ll / dists.mean(axis=-1))
  aux_d = data - np.take(data, indices=[0], axis=-1)
  d = np.max(np.abs(aux_d), axis=-1)
  kfd = np.squeeze(ln / (ln + np.log10(d / ll)))
  if not kfd.ndim:
      kfd = kfd.item()
  return kfd;

Frequency Domain Features

In [9]:
def spectral_entropy(x, sf, method='welch', nperseg=None, normalize=False, axis=-1):
  return ap.spectral_entropy(x,sf,method,nperseg,normalize,axis);

Average Band Power

In [10]:
def bandpower(data,sf,band,window_sec=None,relative=False):#original code from https://raphaelvallat.com/bandpower.html
     band=np.asarray(band);
     low,high=band;
     if window_sec is not None:
       nperseg=window_sec*sf;
     else:
       nperseg=(2/low)*sf;
     freqs,psd=welch(data,sf,nperseg=nperseg)
     freqs_res=freqs[1]-freqs[0];
     idx_band=np.logical_and(freqs>=low,freqs<=high)
     bp=simps(psd[idx_band],dx=freqs_res)

     if relative:
       bp/=simps(psd,dx=freqs_res)
     return bp;

In [11]:
def calc_bandpower(data,band):
  data1=copy.deepcopy(data)
  for i in range(data1.shape[0]):
    for j in range(data1.shape[1]):
      bp=bandpower(data1[i][j],128,band);
      data1[i][j][0]=bp;
  data1=data1[:,:,0];
  print(data1);
  return data1;

Delta Band Power(Fmin=0.5,Fmax=4). Typically related to deep sleep (dreamless)

In [12]:
delta_bp=calc_bandpower(datax,(0.5,4));

[[0.00205205 0.00453002 0.00492838 ... 0.00341536 0.00624304 0.0044921 ]
 [0.00501522 0.00814587 0.00903545 ... 0.00161523 0.01335285 0.00572907]
 [0.00220756 0.00317756 0.00496043 ... 0.00138275 0.00432159 0.00252977]
 ...
 [0.00030836 0.02012138 0.00027472 ... 0.00760051 0.08305198 0.00440099]
 [0.00031684 0.01673892 0.00068028 ... 0.00690888 0.06310327 0.0030533 ]
 [0.00016986 0.01401136 0.00047914 ... 0.00673279 0.07752086 0.00402427]]


Theta Band Power(Fmin=4 and Fmax=8). Related to subconcious activity.

In [13]:
theta_bp=calc_bandpower(datax,(4,8));

[[0.2597402  0.32962222 0.37476629 ... 0.1872867  0.44242346 0.36051146]
 [0.28324378 0.37942818 0.40696158 ... 0.17235792 0.54329531 0.42663495]
 [0.23550126 0.30228186 0.35253358 ... 0.16377201 0.4380097  0.34026869]
 ...
 [0.01037554 0.42586705 0.01564588 ... 0.19024814 1.81292716 0.0978196 ]
 [0.01868355 0.35715047 0.02912563 ... 0.14810923 1.56586911 0.08664536]
 [0.01098146 0.45560795 0.0176916  ... 0.18125766 1.74399071 0.09839971]]


Alpha Band Power(Fmin=8 and Fmax=12). Related to a relaxed but concious state

In [14]:
alpha_bp=calc_bandpower(datax,(8,12));

[[0.2101994  0.24968357 0.2855789  ... 0.21429683 0.39229899 0.3382431 ]
 [0.20902878 0.24535741 0.26590626 ... 0.1939311  0.40411598 0.36578028]
 [0.22264904 0.2535818  0.27313083 ... 0.20901967 0.44151411 0.40828417]
 ...
 [0.00740259 0.1211798  0.00975636 ... 0.05791736 0.46942722 0.03199758]
 [0.0146004  0.12088863 0.01711164 ... 0.05332798 0.45961855 0.03482549]
 [0.00811242 0.14155455 0.01082338 ... 0.05982118 0.51555337 0.03443845]]


Beta Band Power(Fmin=12,Fmax=30).Typically means an active and highly concentrated state

In [15]:
beta_bp=calc_bandpower(datax,(12,30)); 

[[0.35603383 0.41146519 0.45357624 ... 0.27308112 0.4396875  0.38965223]
 [0.33201838 0.38487539 0.42450645 ... 0.26717995 0.41568302 0.36664392]
 [0.35548217 0.42218243 0.46825204 ... 0.27870012 0.47054109 0.43418372]
 ...
 [0.01106371 0.06232282 0.01299109 ... 0.03392964 0.20253663 0.0268503 ]
 [0.03600132 0.08815986 0.02114795 ... 0.04078716 0.26148094 0.03277376]
 [0.01219787 0.06892338 0.01305255 ... 0.03730099 0.2078753  0.02286423]]


Gamma Band Power(Fmin=30,Fmax=45) # Hyperactivity. As the participants are watching videos the chance of hyperactivity is less and so we get the following result.

In [16]:
gamma_bp=calc_bandpower(datax,(30,45));

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


We dont add the gamma band as it a zero value array

In [17]:
def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kkurtosis(data),calc_entropy(data),calc_IQR(data),hjorth_activity(data),hjorth_mobility(data)
                          ,hjorth_complexity(data),petrosian_fractD(data),katz_fd(data),spectral_entropy(data,128)),axis=-1);

In [20]:
from tqdm.notebook import tqdm
features=[]
for data in tqdm(datax):
    features.append(concatenate_features(data))
features=np.concatenate((features,delta_bp,theta_bp,alpha_bp,beta_bp),axis=-1);
features=np.array(features)

  0%|          | 0/1280 [00:00<?, ?it/s]

In [21]:
features.shape

(1280, 768)

Some Exploratory Data Analysis

In [22]:
df_=pd.DataFrame(features);

In [23]:
df_.describe()

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
count,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,...,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000,1280.000000
mean,-0.000133,-0.000188,-0.001124,0.000159,-0.000334,-0.000118,-0.001420,0.001124,0.000909,-0.000308,...,0.184439,0.149456,0.112257,0.170583,0.133312,0.115381,0.138355,0.122505,0.141466,0.112568
std,0.023334,0.022212,0.018275,0.024610,0.020372,0.018881,0.019350,0.025276,0.023899,0.017168,...,0.303852,0.139919,0.139559,0.136508,0.121611,0.091673,0.130485,0.103608,0.099578,0.101316
min,-0.165535,-0.146965,-0.187854,-0.202465,-0.106386,-0.137897,-0.195172,-0.143267,-0.234398,-0.117670,...,0.001740,0.001693,0.002714,0.006499,0.007668,0.004006,0.003946,0.001656,0.007924,0.003831
25%,-0.006170,-0.005897,-0.005919,-0.006794,-0.005631,-0.005376,-0.006142,-0.007081,-0.004956,-0.005932,...,0.043268,0.033236,0.021517,0.067966,0.042473,0.041444,0.041365,0.042133,0.063736,0.030359
50%,-0.000174,0.000158,-0.000353,-0.000279,0.000032,0.000106,-0.000319,-0.000001,0.000030,-0.000063,...,0.090991,0.088261,0.048750,0.127960,0.090647,0.094812,0.087091,0.099946,0.118451,0.082270
75%,0.005121,0.005285,0.004408,0.006358,0.004798,0.005420,0.004684,0.007482,0.005383,0.005681,...,0.196175,0.242975,0.138175,0.247908,0.184447,0.166996,0.205641,0.164054,0.202577,0.166952
max,0.251534,0.216404,0.140089,0.224878,0.193328,0.146430,0.133649,0.168540,0.181113,0.151643,...,2.809871,0.578552,0.787711,1.193602,0.625652,0.617530,0.661375,0.541905,0.522362,0.488013


In [27]:
np.max(features)

19643.307080257415

In [28]:
np.min(features)

-inf

We do not want an outlier like -inf to cause problems to our prediction model so we remove it

In [29]:
features[features<-1e300]=-2e6;

Through early data exploration we can infer one of the main problems of our dataset after feature extraction is that the number of training examples is quite significantly greater than the number of features. Therefore now we try adding more features by inserting polynomial features in our dataset.

In [31]:
transform=PolynomialFeatures(degree=2);#degree can be further tuned as a hyperparameter
features=transform.fit_transform(features);

In [32]:
features.shape

(1280, 296065)

In [33]:
np.save('/content/drive/MyDrive/DEAP/extracted_features/' + 'features.npy', features)